# Soft-VC demo

In [1]:
import torch
import torchaudio
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present
import IPython.display as display

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
hubert   = torch.hub.load("bshall/hubert:main",          "hubert_soft").to(device)
acoustic = torch.hub.load("bshall/acoustic-model:main",  "hubert_soft").to(device)
hifigan  = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").to(device)

Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hubert_main
Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_acoustic-model_main
Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hifigan_main


Removing weight norm...


In [3]:
## speech
#vbank = 'ljspeech'
#vbank = 'databaker'
## vocal
vbank = 'sou'
#vbank = 'len'
#vbank = 'lemi'
#vbank = 'hana'
#vbank = 'ema'
#vbank = 'urushi'
#vbank = 'lansi'
## instrumental
#vbank = 'piano'
## misc
#vbank = 'mix'

steps = 'best'
#steps = 500

ckpt = torch.load(f'out/{vbank}/model-{steps}.pt', map_location=device)
consume_prefix_in_state_dict_if_present(ckpt["acoustic-model"], "module.")
acoustic.load_state_dict(ckpt["acoustic-model"])

<All keys matched successfully>

In [4]:
## speech
#wav_fp = 'test/LJ001-0001.wav'
#wav_fp = 'test/000001.wav'
#wav_fp = 'test/BAC009S0724W0121.wav'
#wav_fp = 'test/SSB00050353.wav'
#wav_fp = 'test/common_voice_ab_19904194.wav'
#wav_fp = 'test/000df995-51ab-4a1d-852c-c288e0300bf7.wav'
## vocal (recllst)
#wav_fp = 'test/D#4_しゅしゅししょしゃしょし.wav'
#wav_fp = 'test/_べべぶぼべぼぼ.wav'
#wav_fp = 'test/_きゅんきゅきゅきぇきゅきょきゅ.wav'
#wav_fp = 'test/kan_kan_kan.wav'
## vocal (song)
#wav_fp = 'test/[Hana] FLOWER_1_歌声-トラック_tune-8.wav'
#wav_fp = 'test/[Ema] さがり花_1_歌声-トラック-6.wav'
wav_fp = 'test/[Len] テロル_vocal-4.wav'
## instrumental
#wav_fp = 'test/Ⅱ 小快板-14.wav'
#wav_fp = 'test/十二音小夜曲-10.wav'

source, sr = torchaudio.load(wav_fp)
source = torchaudio.functional.resample(source, sr, 16000)
source = source.unsqueeze(0).to(device)

print('original:')
display.Audio(source.squeeze().cpu(), rate=16000)

original:


In [5]:
with torch.inference_mode():
  # Extract speech units
  units = hubert.units(source)
  # Generate target spectrogram
  mel = acoustic.generate(units).transpose(1, 2)
  # Generate audio waveform
  target = hifigan(mel)

print('converted:')
display.Audio(target.squeeze().cpu(), rate=16000)

converted:


In [9]:
from scipy.io import wavfile
wavfile.write('converted.wav', 16000, target.squeeze().cpu().numpy())